__BIOBSS - Respiratory Analysis__

_This notebook includes guidelines to help using BIOBSS for respiratory signal extraction and respiratory rate estimation from PPG or ECG signal._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#help(biobss)

# Table of Contents
1.[PPG Sample Data](#sampledata)<br>
2.[Preprocessing](#resp_pre)<br>
3.[Extraction of Respiratory Signals](#resp_sig)<br>
4.[Respiratory Signal Filtering](#filt_resp)<br>
5.[Calculation of Respiratory Quality Indices](#resp_rqi)<br>
6.[Respiratory Rate Estimation](#resp_rr)<br>
7.[Fusion of Respiratory Rates](#resp_fusion)<br>

### __PPG Sample Data__
<a id="sampledata"></a>

In [2]:
#Load the sample data. The sample data file contains 100 10-s PPG segments. The samling rate of the segments is 64 Hz. 

data_dir=r'C:\Users\ipek.karakus\Desktop\biobss\sample_data'

filename='ppg_sample_data.csv'

data=pd.read_csv(os.path.join(data_dir,filename), header=None)
sig=np.asarray(data.iloc[0,:]) #The first segment is selected to be used as sample signal. 
fs=64
L=10

### __Preprocessing__
<a id="resp_pre"></a>

In [3]:
#For respiratory rate estimation, the ppg signal should be filtered before extracting respiratory signal
filt_sig=biobss.resptools.elim_vlf(sig,fs)
filt_sig=biobss.resptools.elim_vhf(sig,fs)

In [4]:
#Next, the PPG signal peaks and onsets are detected
info=biobss.preprocess.peak_detection(sig,fs,'peakdet',delta=0.01)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

In [5]:
#In order to calculate the features correctly, the signal segment should start and end with an onset.  
#Additionally, there should be a single peak between consecutive onsets.

info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

### __Extraction of Respiratory Signals__
<a id="resp_sig"></a>

In [6]:
#The next step is to extract the respiratory signal from the PPG signal.

info=biobss.resptools.extract_resp_sig(locs_peaks,peaks,onsets,fs,['AM','FM','BW'],10)

#The method above extracts the respiratory signals based on three modulation types seperately. To select the signal for a specific method.

y_am=info['am_y']
x_am=info['am_x']
y_fm=info['fm_y']
x_fm=info['fm_x']
y_bw=info['bw_y']
x_bw=info['bw_x']

### __Respiratory Signal Filtering__
<a id="filt_resp"></a>

In [7]:
#If required, the extracted respiratory signal can be filtered prior to respiratory rate estimation.

info=biobss.resptools.filter_resp_sig(resampling_rate=10, am_sig=y_am, am_x=x_am, fm_sig=y_fm, fm_x=x_fm, bw_sig=y_bw, bw_x=x_bw)

y_am=info['am_sig']
x_am=info['am_x']
y_fm=info['fm_sig']
x_fm=info['fm_x']
y_bw=info['bw_sig']
x_bw=info['bw_x']

### __Calculation of Respiratory Quality Indices__
<a id="resp_rqi"></a>

In [8]:
#If required, the respiratory quality assessment can be applied. Autocorrelation or Hjorth approaches can be used. 
#Note that, this step should be applied for a specific modulation type.

rqi_am=biobss.resptools.calc_rqi(y_am,resampling_rate=10)
rqi_fm=biobss.resptools.calc_rqi(y_fm,resampling_rate=10)
rqi_bw=biobss.resptools.calc_rqi(y_bw,resampling_rate=10)

C:\Users\ipek.karakus\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)


### __Respiratory Rate Estimation__
<a id="resp_rr"></a>

In [9]:
#Now, the respiratory rate can be estimated. Note that, this method should be used for a specific modulation type.

rr_am=biobss.resptools.estimate_rr(y_am,10,method='peakdet')
rr_fm=biobss.resptools.estimate_rr(y_fm,10,method='peakdet')
rr_bw=biobss.resptools.estimate_rr(y_bw,10,method='peakdet')


### __Fusion of Respiratory Rates__
<a id="resp_fusion"></a>

In [10]:
#Seperate respiratory rates can be fused into a single value. SmartFusion method does not take RQIs into consideration.

rr_fused=biobss.resptools.fuse_rr(fusion_method='SmartFusion', rqi=None, rr_am=rr_am, rr_fm=rr_fm, rr_bw=rr_bw)


In [13]:
#In order to calculate the weighted average for fusion of RRs, the fusion_method should be selected as 'QualityFusion'.
#RQIs (array) should be provided as a keyworded argument. The order of elements in the array should match with the order RRs of different modulation types.
rqi=[rqi_am['hjorth'],rqi_fm['hjorth'],rqi_bw['hjorth']]
rr_fused=biobss.resptools.fuse_rr(fusion_method='QualityFusion', rqi=rqi, rr_am=rr_am, rr_fm=rr_fm, rr_bw=rr_bw)